In [1]:
import numpy as np
import tensorflow as tf
import os
from os import path

In [2]:
def load_embeddings_and_ids(dirpath, embedding_file, ids_file):
    embeddings = np.load(path.join(dirpath, embedding_file))
    with open(path.join(dirpath, ids_file)) as f:
        ids = [int(x) for x in f.readlines()]
        id2index = { _id:i for i,_id in enumerate(ids) }    
    assert (embeddings.shape[0] == len(ids))
    return embeddings, ids, id2index

In [3]:
resnet50_embeddings,\
resnet50_ids,\
resnet50_id2index = load_embeddings_and_ids(
'/mnt/workspace/Ugallery/ResNet50/', 'flatten_1.npy', 'ids')

In [4]:
class Network:
    def __init__(self):
        
        # --- placeholder
        self._pretrained_embeddings = tf.placeholder(shape=[None, 2048], dtype=tf.float32,
                                                     name='pretrained_embeddings')
        # ---- item vectors
        self._item_vectors = self.trainable_item_embedding(self._pretrained_embeddings)
        
    @staticmethod
    def trainable_item_embedding(X):
        with tf.variable_scope("trainable_item_embedding", reuse=tf.AUTO_REUSE):
            fc1 = tf.layers.dense( # None -> 256
                inputs=X,
                units=256,
                activation=tf.nn.selu,
                name='fc1'
            )
            fc2 = tf.layers.dense( # 256 -> 128
                inputs=fc1,
                units=128,
                activation=tf.nn.selu,
                name='fc2'
            )
            return fc2
    
    def get_embedding_vectors(self, sess, pretrained_embeddings):
        return sess.run(self._item_vectors, feed_dict={
            self._pretrained_embeddings: pretrained_embeddings,
        })

In [5]:
MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/youtube_like/v1/'

In [6]:
with tf.Graph().as_default():
    network = Network()  
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        item_embeddings = network.get_embedding_vectors(sess, resnet50_embeddings)

INFO:tensorflow:Restoring parameters from /mnt/workspace/pamessina_models/ugallery/youtube_like/v1/


In [7]:
item_embeddings.shape

(13297, 128)

In [8]:
dir_path = "/mnt/workspace/Ugallery/Youtube-like/v1/"
os.makedirs(dir_path, exist_ok=True)
item_embeddings.dump(dir_path + "embeddings.npy")
with open(dir_path + 'ids', 'w') as f:
    for _id in resnet50_ids:
        f.write('%d\n' % _id)